## Import Required Packages

In [1]:
import sys
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

import time
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

%load_ext tensorboard

from sklearn.utils import shuffle

import numpy.random as rng
np.random.seed(1337)

import cv2

ModuleNotFoundError: No module named 'tfa'

## Create Data Generators

In [2]:
SIZE = 256
IMG_SIZE = (SIZE, SIZE)
NUM_CHANNELS = 3
BATCH_SIZE = 16

# Define training augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #zca_whitening=True,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    vertical_flip=True)

# Define validation augmentations
val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Create example augmented images
img = load_img('F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Training/All Cases/Correct/Correct/IMG_1500.jpg')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/preview', save_format='jpeg'):
    i+=1
    if i > 9:
        break

In [3]:
train_generator = train_datagen.flow_from_directory(
    'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Training/All Cases/Both',
    #'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Training/Edge Cases/Both',
    target_size=IMG_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed=1,
    batch_size=BATCH_SIZE,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Validation/All Cases/Both',
    #'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Binary/Validation/Edge Cases/Both',
    target_size=IMG_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed=2,
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 100 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


## Create Convolutional Neural Network with VGG16 as Base Model

In [4]:
base_logdir = "logs/scalars/"
for f in os.listdir(base_logdir):
    file_path = os.path.join(base_logdir, f)
    
    shutil.rmtree(file_path)

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [5]:
model = VGG16(include_top=False, input_shape=(SIZE, SIZE, NUM_CHANNELS))

# Only retrain last block of VGG16 - 3 conv layers
for layer in model.layers:
    #if not layer.name.startswith('block5'):
    layer.trainable = False

# Add own layers at end for task
flat1 = Flatten()(model.layers[-1].output)
dense1 = Dense(128, activation='relu', kernel_regularizer=l2(1E-4), bias_regularizer=l2(1E-3))(flat1)
dropout1 = Dropout(0.5)(dense1)

output = Dense(1, activation='sigmoid')(dropout1)

model = Model(inputs=model.inputs, outputs=output)

model.summary()

optimizer = Adam(lr=0.0001)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy', 'Precision', 'Recall'])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)      

In [6]:
model.fit(train_generator, steps_per_epoch=train_generator.samples//BATCH_SIZE, 
          validation_data=validation_generator, validation_steps=validation_generator.samples//BATCH_SIZE,
          epochs=25,
#           class_weight={0: 1, 1: 3},
          callbacks=[tensorboard_callback], verbose=1)

Epoch 1/25
1/6 [====>.........................] - ETA: 0s - loss: 0.6769 - accuracy: 0.6250 - precision: 0.0000e+00 - recall: 0.0000e+00WARNING:tensorflow:From f:\work\university\year 5\acs6420_advanced project\code\siamese_env\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
6/6 [==============================] - 14s 2s/step - loss: 0.8472 - accuracy: 0.6071 - precision: 0.6538 - recall: 0.4146 - val_loss: 1.0982 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 2/25
6/6 [==============================] - 8s 1s/step - loss: 0.7314 - accuracy: 0.5714 - precision: 0.5455 - recall: 0.8571 - val_loss: 0.8129 - val_accuracy: 0.5250 - val_precision: 0.6250 - val_recall: 0.1250
Epoch 3/25
6/6 [==============================] - 8s 1s/step - loss: 0.5366 - accuracy: 0.6905 - precision: 0.9

In [8]:
%tensorboard --logdir logs/scalars

Reusing TensorBoard on port 6006 (pid 15136), started 0:01:16 ago. (Use '!kill 15136' to kill it.)

In [20]:
model_json = model.to_json()
modeldir = "models/post_process/CNN/structure/" + train_time
weightdir = "models/post_process/CNN/weights/" + train_time

with open(modeldir + ".json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights(weightdir + ".h5")

## Test Performance on Each Category

In [24]:
test_path_root = 'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/Post Installation/Multiclass'
test_category = '/Outside Guide_Incorrect Adhesion 2'
test_path = test_path_root + test_category

img_list = []
for img in os.listdir(test_path):
    img_array = cv2.imread(os.path.join(test_path, img))
    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
    img_array = cv2.resize(img_array, IMG_SIZE)
    img_array = img_array/255
    img_list.append(img_array)

In [25]:
for img in img_list:
    X = np.reshape(img, (1, SIZE, SIZE, NUM_CHANNELS))
    prediction = model.predict(X)
    print(round(prediction[-1][-1]), '\t', prediction[-1][-1])

1.0 	 1.0
1.0 	 1.0
1.0 	 1.0
1.0 	 1.0
1.0 	 1.0
1.0 	 1.0
1.0 	 0.9999995
1.0 	 1.0
1.0 	 1.0
1.0 	 1.0
1.0 	 0.76496613
0.0 	 0.07216001
1.0 	 0.9996753
